In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Load the dataset
url = 'https://example.com/bitcoin-historical-data.csv'  # Replace with a link to a dataset or your local path
df = pd.read_csv(url)
df.head()


In [ ]:
# Display dataset information and check for missing values
print(df.info())
print(df.isnull().sum())

# Convert date column to datetime format if necessary
df['Date'] = pd.to_datetime(df['Date'])

# Sort by date to ensure proper time order
df = df.sort_values('Date')

# Select features for training, e.g., Close price
data = df[['Close']].values  # Using only the 'Close' price for simplicity


In [ ]:
# Use MinMaxScaler for normalization
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data)


In [ ]:
def create_sequences(data, sequence_length=60):
    x, y = [], []
    for i in range(sequence_length, len(data)):
        x.append(data[i-sequence_length:i, 0])
        y.append(data[i, 0])
    return np.array(x), np.array(y)

# Create sequences with a 60-day lookback period
sequence_length = 60
x, y = create_sequences(scaled_data, sequence_length)

# Reshape data to (samples, time steps, features) format required by LSTM
x = np.reshape(x, (x.shape[0], x.shape[1], 1))


In [ ]:
# Split into training (80%) and testing (20%) sets
split = int(0.8 * len(x))
x_train, x_test = x[:split], x[split:]
y_train, y_test = y[:split], y[split:]


In [ ]:
# Define the LSTM model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(x_train.shape[1], 1)))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(units=25))
model.add(Dense(units=1))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Display the model summary
model.summary()


In [ ]:
# Add early stopping to prevent overfitting
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model
history = model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=50, batch_size=32, callbacks=[early_stop])


In [ ]:
# Predict using the test set
predictions = model.predict(x_test)

# Inverse transform the predictions to original scale
predictions = scaler.inverse_transform(predictions)
y_test_scaled = scaler.inverse_transform([y_test])

# Calculate error metrics
mse = mean_squared_error(y_test_scaled[0], predictions[:, 0])
mae = mean_absolute_error(y_test_scaled[0], predictions[:, 0])
print(f'Mean Squared Error: {mse}')
print(f'Mean Absolute Error: {mae}')


In [ ]:
# Prepare a dataframe for plotting
train = df[:split + sequence_length]
test = df[split + sequence_length:]
test['Predictions'] = predictions

# Plot the actual and predicted prices
plt.figure(figsize=(16, 8))
plt.plot(train['Date'], train['Close'], label='Train Data')
plt.plot(test['Date'], test['Close'], label='Actual Price')
plt.plot(test['Date'], test['Predictions'], label='Predicted Price')
plt.title('Bitcoin Price Prediction')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()
plt.show()


In [ ]:
# Save the model for future use
model.save('bitcoin_lstm_model.h5')
